In [ ]:
import cobra
from cobra import Model, Reaction, Metabolite
import re


In [ ]:
WormJam = cobra.io.read_sbml_model('/path_to_dir/WormJam-GEM.xml') #Import WormJam

In [ ]:
WormJam.objective = 'Biomass2'

In [ ]:
file_cel_tm = open('/path_to_dir/tm_cel_orthologs.txt','r')#Import the biomart file that contains T. muris and C. elegans orthologs

celegans_tmuris_orthologs_o2o = {}
only_celegans_orthologs = []

for i in file_cel_tm:
    try:
        if str(i.strip().split('\t')[-1]) == 'ortholog_one2one':
            celegans_tmuris_orthologs_o2o[str(i.strip().split('\t')[1])] = i.strip().split('\t')[0]
            only_celegans_orthologs.append(i.strip().split('\t')[1])
    except Exception:
        pass

In [ ]:
len(celegans_tmuris_orthologs_o2o)

In [ ]:
i = 0
tmuris_generactionrule = {}
while i < len(WormJam.reactions):
    if 'WBGene' in str(WormJam.reactions[i].gene_reaction_rule):
        tmuris_generactionrule[str(WormJam.reactions[i].id)] = str(WormJam.reactions[i].gene_reaction_rule)
    else:
        tmuris_generactionrule[str(WormJam.reactions[i].id)] = 'no_gene_association'
    
    i += 1

In [ ]:
tmuris_generactionrule

In [ ]:
for i in tmuris_generactionrule:
    for l in range(len(tmuris_generactionrule[i].split(' '))):
        if 'WBGene' in tmuris_generactionrule[i].split(' ')[l]:
            if tmuris_generactionrule[i].split(' ')[l].strip('()') in only_celegans_orthologs:
                tmuris_generactionrule[i] = str(tmuris_generactionrule[i]).replace(tmuris_generactionrule[i].split(' ')[l].strip('()'),celegans_tmuris_orthologs_o2o[str(tmuris_generactionrule[i].split(' ')[l].strip('()'))])
            elif tmuris_generactionrule[i].split(' ')[l].strip('()') not in only_celegans_orthologs:
                tmuris_generactionrule[i] = str(tmuris_generactionrule[i]).replace(tmuris_generactionrule[i].split(' ')[l].strip('()'), 'no_ortholog')
            

In [ ]:
tmuris_generactionrule

In [ ]:
for i in tmuris_generactionrule:
    if 'and' not in tmuris_generactionrule[i] and 'TMUE' in tmuris_generactionrule[i]:
        tmuris_generactionrule[i] = (' or '.join(re.findall('TMUE_\d+', tmuris_generactionrule[i])))

In [ ]:
tmuris_generactionrule

In [ ]:
draft_model_tmuris = Model('T. muris draft V1')
for i in tmuris_generactionrule:
    if 'TMUE' in tmuris_generactionrule[i] or 'no_gene_association' in tmuris_generactionrule[i]:
        if 'no_ortholog' not in tmuris_generactionrule[i]:
            if 'n' not in WormJam.reactions.get_by_id(str(i)).compartments:
                if 'no_gene_association' in tmuris_generactionrule[i]:
                    if len(WormJam.reactions.get_by_id(str(i)).compartments) >= 2 or 'e' in WormJam.reactions.get_by_id(str(i)).compartments:
                        draft_model_tmuris.add_reactions([WormJam.reactions.get_by_id(str(i))])
                        draft_model_tmuris.reactions.get_by_id(str(i)).gene_reaction_rule = tmuris_generactionrule[i]
                else:
                    draft_model_tmuris.add_reactions([WormJam.reactions.get_by_id(str(i))])
                    draft_model_tmuris.reactions.get_by_id(str(i)).gene_reaction_rule = tmuris_generactionrule[i]

In [ ]:
cobra.io.save_json_model(draft_model_tmuris, "Draft_Tmuris_V1.json")
cobra.io.save_matlab_model(draft_model_tmuris, "Draft_Tmuris_V1.mat")
cobra.io.write_sbml_model(draft_model_tmuris, "Draft_Tmuris_V1.xml")